# 📓 Notebook Databricks : merger les CSV et créer une table Delta

# 🔧 Étape 1 – Config (stockage et chemin CSV)

In [0]:
storage_account_name = "comptestockage001"
container_name = "adfv2tutorial"
conf_key = f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net"

# Récupération de la clé
storage_key = dbutils.secrets.get(scope="blobstorage_scope", key="key3")

# Configuration Spark
spark.conf.set(conf_key, storage_key)

# Chemin vers le conteneur
wasbs_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/"

# Chemin vers les fichiers CSV (tu peux adapter le dossier)
csv_folder = wasbs_path

# 📂 Étape 2 – Lire et fusionner les fichiers .csv
## 2.0 Contrôle de schéma avant fusion

In [0]:
# Lister tous les fichiers .csv dans le dossier
files = dbutils.fs.ls(csv_folder)
csv_files = [f.path for f in files if f.path.endswith(".csv")]

print(f"{len(csv_files)} fichiers CSV trouvés")

# Lire tous les fichiers et vérifier la cohérence des schémas
schemas = {}
df_all = None

for path in csv_files:
    df = spark.read.option("header", True).option("inferSchema", True).csv(path)
    current_schema = set((field.name, field.dataType.simpleString()) for field in df.schema.fields)
    
    # Stocker le schéma du premier fichier comme référence
    if df_all is None:
        reference_schema = current_schema
        df_all = df
        schemas[path] = "OK"
    else:
        if current_schema == reference_schema:
            df_all = df_all.unionByName(df)
            schemas[path] = "OK"
        else:
            schemas[path] = "SCHÉMA NON CONFORME"
            print(f" Fichier ignoré (schéma différent) : {path}")

# Résumé des statuts de fichiers
print("\nRésumé de l'analyse des schémas :")
for file_path, status in schemas.items():
    print(f"- {file_path} : {status}")

# Afficher les données fusionnées
if df_all is not None:
    display(df_all)
else:
    print("Aucun fichier valide n’a été fusionné.")



# Étape 3 – Sauvegarde dans une table Delta

In [0]:
df_all.write.mode("overwrite").format("delta").saveAsTable("table_csv_fusionnee")